# Test with baseline model

In [4]:
!pip install dataset accelerate databench_eval

In [5]:

# import pandas as pd

# # Example DataFrame
# data = {
#     'Name': ['Alice', 'Bob', 'Charlie'],
#     'Age': [25, 30, 35],
#     'Salary': [50000.0, 60000.0, 70000.0],
#     'Is_Employed': [True, False, True]
# }
# # What is the average salary of employees who are employed and older than 28?
# df = pd.DataFrame(data)
# return df[(df['Is_Employed'] == True) & (df['Age'] > 28)]['Salary'].mean()



In [6]:
import pandas as pd
import subprocess
import shlex
import zipfile
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from databench_eval import Runner, Evaluator, utils

### set up

In [7]:
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b")
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stable-code-3b",
  torch_dtype="auto",
)
model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/610M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affin

In [8]:
def call_model(prompts):
  """
  tokenize prompt, model generate;
  """
  results = []
  for p in prompts:
      inputs = tokenizer(p, return_tensors="pt").to(model.device)
      tokens = model.generate(
          **inputs,
          max_new_tokens=128,
          temperature=0.2,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
      result = tokenizer.decode(tokens[0], skip_special_tokens=True)
      results.append(result)
  return results

In [9]:
# load data and evaluator
qa = utils.load_qa(name="semeval", split="dev")
evaluator = Evaluator(qa=qa)

README.md:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

qa.parquet:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/6.71k [00:00<?, ?B/s]

...

qa.parquet:   0%|          | 0.00/7.18k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/8.25k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/8.73k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/988 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/320 [00:00<?, ? examples/s]

In [10]:
qa[0]

{'question': 'Is the most favorited author mainly communicating in Spanish?',
 'answer': 'True',
 'type': 'boolean',
 'columns_used': "['favorites', 'lang']",
 'column_types': "['category', 'category']",
 'sample_answer': 'True',
 'dataset': '050_ING'}

In [11]:
qa

Dataset({
    features: ['question', 'answer', 'type', 'columns_used', 'column_types', 'sample_answer', 'dataset'],
    num_rows: 320
})

### overridden compare in Evaluator

In [12]:
# reason: qa[211] always throw error message:
# Semantic not supported:  category
# qa[211]['type']= ' category', the string start with space;
# With the default evaluation see code line 21, elif semantic == "category"
# this line result the evaluator always throw Exception(f"Semantic not supported: {semantic}")

In [13]:
err_dict,none_str_dict = {},{}
for i in range(len(qa)):
   for value_ in qa[i].values():
    if isinstance(value_, str):
      if value_.startswith(" "):
        err_dict[i]=qa[i]
    else:
      none_str_dict[i]=qa[i]
err_dict # see index 211

{191: {'question': 'In which province are the most cars located?',
  'answer': ' Madrid',
  'type': 'category',
  'columns_used': "['province']",
  'column_types': "['category']",
  'sample_answer': 'Madrid',
  'dataset': '059_Second'},
 209: {'question': "On weekdays, what's the most commonly bought item?",
  'answer': 'Coffee',
  'type': 'category',
  'columns_used': '[Item, weekday_weekend]',
  'column_types': "['category', 'category']",
  'sample_answer': ' Coffee',
  'dataset': '060_Bakery'},
 211: {'question': 'During which period of the day is Brownie" most frequently bought?"',
  'answer': ' afternoon',
  'type': 'category',
  'columns_used': '[Item, period_day]',
  'column_types': "['category', 'category']",
  'sample_answer': ' afternoon',
  'dataset': '060_Bakery'}}

In [14]:
def custom_compare(value, truth, semantic):
  # override the default compare method;
    semantic = semantic.strip()  # add this line to remove the space;

    if semantic == "boolean":
        return str(value) == str(truth)
    elif semantic == "category":
        return str(value) == str(truth)
    elif semantic == "number":
        try:
            value_cleaned = ''.join(char for char in str(value) if char.isdigit() or char in ['.', '-'])
            truth_cleaned = ''.join(char for char in str(truth) if char.isdigit() or char in ['.', '-'])
            return round(float(value_cleaned), 2) == round(float(truth_cleaned), 2)
        except :
            return False
    elif semantic == "list[category]":
        try:
            value_list = [item.strip() for item in str(value).strip('[]').split(',')]
            truth_list = [item.strip() for item in str(truth).strip('[]').split(',')]
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    elif semantic == "list[number]":
        try:
            value_list = sorted(round(float(''.join(c for c in v.strip() if c.isdigit() or c in ['.', '-'])), 2) for v in str(value).strip('[]').split(',') if v.strip())
            truth_list = sorted(round(float(''.join(c for c in t.strip() if c.isdigit() or c in ['.', '-'])), 2) for t in str(truth).strip('[]').split(',') if t.strip())
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    else:
        raise ValueError(f"Semantic not supported: '{semantic}'")

In [15]:
custom_evaluator = Evaluator(compare=custom_compare,qa=qa)

### response

In [16]:
prompts = [] # prompt
code_res_full = [] # generated code, full text
code_res_run = [] # generated code, trunct till the first \n, what is actually running

In [17]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = utils.load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]



# EXAMPLE 1 : An example of writing a one line code query for the following question: What is the average salary of employees who are employed and older than 28?

def example(df: pd.DataFrame) -> float:
    data = {{
        'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'Salary': [50000.0, 60000.0, 70000.0],
        'Is_Employed': [True, False, True]
    }}
    df = pd.DataFrame(data)
    return df[(df['Is_Employed'] == True) & (df['Age'] > 28)]['Salary'].mean()

# TODO: complete the following function in one line. It should give the answer to: {question}

def answer(df: pd.DataFrame) -> {row["type"]}:
  #first_row_example parameter and column_types are just examples to give you information about  the specified dataframe column types and an example of one row in the dataframe
    df.columns = {list(df.columns)}
    first_row_example ={list(df.iloc[0])}
    column_types = {list(df.dtypes)}
    return"""

    prompts.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full = response.split("return")[2]
        resp_run = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full.append(resp_full) # append to code_responses
        code_res_run.append(resp_run)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [18]:
runner = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, utils.load_table
    ),
    qa=qa,
    batch_size=10,
)

In [19]:
responses = runner.run()

 25%|██▌       | 8/32 [20:48<1:02:26, 156.11s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.18 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1.57 GiB is free. Process 10893 has 13.18 GiB memory in use. Of the allocated memory 10.59 GiB is allocated by PyTorch, and 2.46 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
#print(f"DataBench accuracy is {evaluator.eval(responses)}")  # ~0.15
custom_evaluator.eval(responses)

In [ ]:
df_responses = {
    'prompt': prompts,
    'full_code_response': code_res_full,
    "code_implement":code_res_run,
    'data_ans': responses,
    "gold": qa["answer"]
}

df = pd.DataFrame(df_responses)
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/My Drive/response_stable-code-3b.csv', index=False)

### lite -> to create a lite df

In [ ]:
prompts_lite = [] # prompt
code_res_full_lite = [] # generated code, full text
code_res_run_lite = [] # generated code which is actually running

In [ ]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = utils.load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}

    return"""

    prompts_lite.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full_lite = response.split("return")[2]
        resp_run_lite = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full_lite.append(resp_full_lite)
        code_res_run_lite.append(resp_run_lite)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [ ]:
runner_lite = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, utils.load_sample
    ),
    qa=qa,
    batch_size=10,
)

In [ ]:
responses_lite = runner_lite.run()

In [ ]:
print(
    f"DataBench_lite accuracy is {custom_evaluator.eval(responses_lite, lite=True)}"
) # ~0.07

In [ ]:
df_responses_lite = {
    'prompt': prompts_lite,
    'full_code_response': code_res_full_lite,
    "running_code_response":code_res_run_lite,
    'data_ans': responses_lite,
    "gold": qa["answer"]
}

df_lite = pd.DataFrame(df_responses_lite)
df_lite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_lite.to_csv('/content/drive/My Drive/response_lite_stable-code-3b.csv', index=False)